#  "Understanding Risk Patterns in Diabetes Patients"

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('/kaggle/input/diabetes-data-set/diabetes.csv')

In [ ]:
y = df['Outcome']
X = df.drop('Outcome', axis =1)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
# Check Nulls

X.isnull().sum()

In [ ]:
X.head()

In [ ]:
X.dtypes

In [ ]:
# check outlier

import matplotlib.pyplot as plt
import seaborn as sns

for col in X.columns:
    plt.figure(figsize = (5,6))
    sns.boxplot(X[col])
    plt.title(col)
    plt.show()

In [ ]:
# almost all fetaures have outliers

In [ ]:
y.value_counts()

In [ ]:
# all are integer or float columns , so no processing required in that arena

In [ ]:
# about OUTLIERS!!  and scaling features

In [ ]:
# Droppping rows with outliers is not dropping too may  rows

In [ ]:
Xy = pd.concat([X, y], axis=1)

In [ ]:
mask = pd.Series(True, index=Xy.index)
for col in Xy.columns:
    Q1 = Xy[col].quantile(0.25)
    Q3 = Xy[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    mask &= (Xy[col] >= lower) & (Xy[col] <= upper)

Xy = Xy[mask]

In [ ]:
Xy.shape

In [ ]:
X = Xy.drop('Outcome', axis=1)
y = Xy['Outcome']

In [ ]:
# sacle the data

In [ ]:
for col in X.columns:
    plt.figure(figsize = (5,6))
    sns.histplot(X[col])
    plt.title(col)
    plt.show()

In [ ]:
# Inspect Skin thickness and Insulin

print(X['SkinThickness'].value_counts())

print(X['Insulin'].value_counts())

In [ ]:
# skin thickness / Insulin have large numbe of 0s

In [ ]:
#  as 0 skin is meaningless, replace 0 with nans

X['SkinThickness'] = X['SkinThickness'].replace(0, np.nan)
X['Insulin'] = X['Insulin'].replace(0, np.nan)

In [ ]:
# Impute missing values for these columns
# using KNNimputer

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors = 5)
X[['SkinThickness', 'Insulin']] = imputer.fit_transform(X[['SkinThickness', 'Insulin']])

In [ ]:
# pregnancies and age
# Check skewnewss

X[['Pregnancies', 'Age']].skew()

In [ ]:
# apply log1px transform

X['Pregnancies'] = np.log1p(X['Pregnancies'])
X['Age'] = np.log1p(X['Age'])

In [ ]:
# now apply scaling

from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)
X = pd.DataFrame(X_scaled, columns  = X.columns)

In [ ]:
X.head()

In [ ]:
# Applying Random forest Classifier on the data

In [ ]:
# Random forest classifier + recall
from  sklearn.model_selection  import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import random
from scipy.stats import randint
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

RF = RandomForestClassifier(class_weight='balanced', random_state = 42)

param_dist = {
 'n_estimators': randint(100, 300),
    'max_depth': randint(5, 15),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['auto', 'sqrt', 'log2']    
}


random_search = RandomizedSearchCV(param_distributions = param_dist, 
                                   estimator = RF, cv = 3, n_iter=20,random_state=42,
                                  scoring = 'recall')

random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
print("accuracy for Random forest is ", accuracy_score(y_test, y_pred))

# 2. Classification Report (includes precision, recall, F1-score for each class)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# 3. Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
# XGBoost classifier with recall as (best)
from xgboost import XGBClassifier
from scipy.stats import uniform
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]
XGBclassifier = XGBClassifier(scale_pos_weight=scale_pos_weight, random_state = 42)

param_dist = {
 'n_estimators': randint(100, 500),         
    'max_depth': randint(3, 15),              
    'learning_rate': uniform(0.01, 0.3),       
    'subsample': uniform(0.6, 0.4),            
    'colsample_bytree': uniform(0.6, 0.4),     
    'gamma': uniform(0, 5),                   
    'reg_alpha': uniform(0, 1),               
    'reg_lambda': uniform(0, 1)   
}


random_search = RandomizedSearchCV(param_distributions = param_dist, 
                                   estimator = XGBclassifier, cv = 3, n_iter=20,random_state=42,
                                  scoring = 'recall')

random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
print("accuracy for XGBOOST is ", accuracy_score(y_test, y_pred))

# 2. Classification Report (includes precision, recall, F1-score for each class)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# 3. Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
# to further reduce false negatives - as  this is healthcare problem  ---> did not help

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]

XGBclassifier = XGBClassifier(
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    use_label_encoder=False,
    objective='binary:logistic'
)

# Random search
random_search = RandomizedSearchCV(
    estimator=XGBclassifier,
    param_distributions=param_dist,
    n_iter=20,
    scoring='recall',
    cv=3,
    random_state=42,
    verbose=1
)

# Fit with early stopping using **fit_params
random_search.fit(
    X_train,
    y_train
)

# Best model
best_model = random_search.best_estimator_
best_model.fit(X_train, y_train,
               eval_set=[(X_test, y_test)],
               early_stopping_rounds=10,
               eval_metric='aucpr',
               verbose=False)

y_pred = best_model.predict(X_test)

# Results
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
# threshold tuning

In [ ]:
from sklearn.metrics import f1_score

f1_scores = []
for t in thresholds:
    y_pred_thresh = (y_probs >= t).astype(int)
    f1 = f1_score(y_test, y_pred_thresh)
    f1_scores.append(f1)

best_thresh = thresholds[f1_scores.index(max(f1_scores))]
print("Best threshold for max F1:", round(best_thresh, 2))

In [ ]:
# XGBoost classifier with recall as (best)
from xgboost import XGBClassifier
from scipy.stats import uniform
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.3, random_state = 42)
scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]
XGBclassifier = XGBClassifier(scale_pos_weight=scale_pos_weight, random_state = 42)

param_dist = {
 'n_estimators': randint(100, 500),         
    'max_depth': randint(3, 15),              
    'learning_rate': uniform(0.01, 0.3),       
    'subsample': uniform(0.6, 0.4),            
    'colsample_bytree': uniform(0.6, 0.4),     
    'gamma': uniform(0, 5),                   
    'reg_alpha': uniform(0, 1),               
    'reg_lambda': uniform(0, 1)   
}


random_search = RandomizedSearchCV(param_distributions = param_dist, 
                                   estimator = XGBclassifier, cv = 3, n_iter=20,random_state=42,
                                  scoring = 'recall')

random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_
y_probs = best_model.predict_proba(X_test)[:, 1]

threshold = 0.35
y_pred_thresh = (y_probs >= threshold).astype(int)


print("Confusion Matrix at threshold =", threshold)
print(confusion_matrix(y_test, y_pred_thresh))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_thresh))


# Summary
This project uses the Pima Indians Diabetes dataset to build a robust machine learning pipeline that predicts the presence of diabetes based on diagnostic measurements. Given the medical context, the primary objective is to maximize recall for the positive class (patients with diabetes), minimizing false negatives to avoid missed diagnoses.

**Key Highlights:**
Objective: Binary classification to predict diabetes (0 = No, 1 = Yes)

Dataset: Pima Indians Diabetes Dataset (UCI / Kaggle)

Problem Type: Imbalanced classification (class 1 ~35%)

Primary Metric: Recall (sensitivity) for class 1

**Workflow & Techniques:**

EDA & Preprocessing:

* Handled outliers using IQR method
* Replaced biologically implausible 0 values (e.g., insulin, skin thickness) via KNN imputation
* Applied RobustScaler to handle skewed distributions
* Stratified train-test split to preserve class balance


**Modeling:**

* Compared Random Forest, Logistic Regression, and XGBoost
* Used scale_pos_weight in XGBoost to address imbalance
* Performed RandomizedSearchCV with recall as scoring metric

**Threshold Tuning:**

* Selected threshold to maximize F1-score and recall without compromising precision significantly

**Results:**

* Metric	Value (XGBoost with threshold tuning)
* Accuracy	~78%
* Recall (Class 1)	↑ > 80%
* Precision (Class 1)	~60%
* Confusion Matrix	Included in final notebook

**Key Learnings:**

* Class imbalance can be addressed with a combination of sampling strategies, custom loss weighting, and threshold optimization
* In healthcare problems, recall is more critical than accuracy
* Proper EDA, scaling, and imputation are essential for model stability and reliability
